In [ ]:
!git clone https://github.com/ujwal-jibhkate/lora-vs-ia3.git
%cd lora-vs-ia3

Cloning into 'lora-vs-ia3'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 66 (delta 32), reused 47 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (66/66), 38.66 KiB | 6.44 MiB/s, done.
Resolving deltas: 100% (32/32), done.
/content/lora-vs-ia3


In [ ]:
!pip install -q -r requirements.txt
!pip install -q --upgrade transformers peft evaluate accelerate

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 18.4 MB/s eta 0:00:00


In [ ]:
import sys
sys.path.append('/content/lora-vs-ia3')

In [ ]:
import os
from google.colab import userdata
import wandb

os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
os.environ["WANDB_API_KEY"] = userdata.get('WANDB_API_KEY')
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Currently logged in as: ujwaljibhkate06 (iu-aml-hw4-team) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
DRIVE_SAVE_PATH_BASE = "/content/drive/MyDrive/lora-vs-ia3/adapters" # Make sure this matches your desired path
os.makedirs(DRIVE_SAVE_PATH_BASE, exist_ok=True)

Mounted at /content/drive


In [ ]:
%cd /content/lora-vs-ia3
!git pull origin main

/content/lora-vs-ia3
From https://github.com/ujwal-jibhkate/lora-vs-ia3
 * branch            main       -> FETCH_HEAD
Already up to date.


In [ ]:
from src.trainer import run_experiment
import os

HF_TOKEN = os.getenv("HF_TOKEN")

# Base config for T5 + SAMSum
base_config = {
    "task_name": "samsum",
    "model_name": "t5",
    "num_epochs": 3,
    "batch_size": 4, # Keep this small
    "gradient_accumulation": 4, # Keep this
    "learning_rate": 5e-4,
    "wandb_project": "lora-vs-ia3-t5",

    "train_subset_size": 5000, # Train on 5000 samples instead of 14k+
    "eval_subset_size": 200    # Evaluate on 200 samples instead of 800+
    # -----------------------
}



In [ ]:
for r_val in [8, 16]:
    run_config_lora = base_config.copy()
    run_config_lora.update({
        "run_name": f"t5_samsum_lora_r{r_val}",
        "peft_method": "lora",
        "r": r_val,
        "lora_alpha": r_val * 2,
    })

    print(f"\n--- STARTING LoRA RUN: r={r_val} ---")
    run_experiment(run_config_lora, HF_TOKEN)
    print(f"--- FINISHED LoRA RUN: r={r_val} ---")

    # Save adapter to Drive
    run_name_lora = run_config_lora.get('run_name')
    colab_adapter_path_lora = f"./results/{run_name_lora}/best_adapter"
    drive_adapter_path_lora = os.path.join(DRIVE_SAVE_PATH_BASE, run_name_lora)
    print(f"Copying adapter from {colab_adapter_path_lora} to {drive_adapter_path_lora}...")
    !cp -r "{colab_adapter_path_lora}" "{drive_adapter_path_lora}"
    print("Adapter copied to Google Drive.")


--- STARTING LoRA RUN: r=8 ---


--- Loading dataset: samsum ---
--- Loading SUBSET of 5000 training samples ---


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

...

eval/loss,█▃▁▁
eval/runtime,▇█▁▁
eval/samples_per_second,▂▁██
eval/steps_per_second,▂▁██
train/epoch,▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇█████
train/global_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇███
train/grad_norm,▄ █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▂▅███▇▇▇▇▇▆▆▆▆▆▆▆▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,3
eval/loss,0.388


--- FINISHED LoRA RUN: r=8 ---
Copying adapter from ./results/t5_samsum_lora_r8/best_adapter to /content/drive/MyDrive/lora-vs-ia3/adapters/t5_samsum_lora_r8...
Adapter copied to Google Drive.

--- STARTING LoRA RUN: r=16 ---


--- Loading dataset: samsum ---
--- Loading SUBSET of 5000 training samples ---
Using 5000 training samples.
--- Loading SUBSET of 200 evaluation samples ---
Using 200 eval samples out of 818.
--- Loading model: t5 ---
--- Applying PEFT adapter: lora ---
trainable params: 589,824 || all params: 61,096,448 || trainable%: 0.9654
--- Using Seq2SeqTrainer (for eval_loss only) ---
--- Starting Training: t5_samsum_lora_r16 ---


/content/lora-vs-ia3/src/trainer.py:173: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = TrainerClass(


Epoch,Training Loss,Validation Loss
1,0.478800,0.394291
2,0.473600,0.384573


In [ ]:
for r_val in [16]:
    run_config_lora = base_config.copy()
    run_config_lora.update({
        "run_name": f"t5_samsum_lora_r{r_val}",
        "peft_method": "lora",
        "r": r_val,
        "lora_alpha": r_val * 2,
    })

    print(f"\n--- STARTING LoRA RUN: r={r_val} ---")
    run_experiment(run_config_lora, HF_TOKEN)
    print(f"--- FINISHED LoRA RUN: r={r_val} ---")

    # Save adapter to Drive
    run_name_lora = run_config_lora.get('run_name')
    colab_adapter_path_lora = f"./results/{run_name_lora}/best_adapter"
    drive_adapter_path_lora = os.path.join(DRIVE_SAVE_PATH_BASE, run_name_lora)
    print(f"Copying adapter from {colab_adapter_path_lora} to {drive_adapter_path_lora}...")
    !cp -r "{colab_adapter_path_lora}" "{drive_adapter_path_lora}"
    print("Adapter copied to Google Drive.")


--- STARTING LoRA RUN: r=16 ---


--- Loading dataset: samsum ---
--- Loading SUBSET of 5000 training samples ---
Using 5000 training samples.
--- Loading SUBSET of 200 evaluation samples ---
Using 200 eval samples out of 818.
--- Loading model: t5 ---
--- Applying PEFT adapter: lora ---
trainable params: 589,824 || all params: 61,096,448 || trainable%: 0.9654
--- Using Seq2SeqTrainer (for eval_loss only) ---
--- Starting Training: t5_samsum_lora_r16 ---


/content/lora-vs-ia3/src/trainer.py:173: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = TrainerClass(


Epoch,Training Loss,Validation Loss
1,0.478800,0.394291
2,0.473600,0.384573
3,0.477900,0.381898


--- Starting Evaluation (calculating eval_loss) ---


Evaluation Results: {'eval_loss': 0.38189783692359924, 'eval_runtime': 10.5268, 'eval_samples_per_second': 18.999, 'eval_steps_per_second': 9.5, 'epoch': 3.0}
--- Saving Adapter ---
Best adapter saved to ./results/t5_samsum_lora_r16/best_adapter


eval/loss,█▃▁▁
eval/runtime,██▅▁
eval/samples_per_second,▁▁▄█
eval/steps_per_second,▁▁▄█
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇███
train/global_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇██
train/grad_norm,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,████▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,3
eval/loss,0.3819


--- FINISHED LoRA RUN: r=16 ---
Copying adapter from ./results/t5_samsum_lora_r16/best_adapter to /content/drive/MyDrive/lora-vs-ia3/adapters/t5_samsum_lora_r16...
Adapter copied to Google Drive.


In [ ]:
for r_val in [32]:
    run_config_lora = base_config.copy()
    run_config_lora.update({
        "run_name": f"t5_samsum_lora_r{r_val}",
        "peft_method": "lora",
        "r": r_val,
        "lora_alpha": r_val * 2,
    })

    print(f"\n--- STARTING LoRA RUN: r={r_val} ---")
    run_experiment(run_config_lora, HF_TOKEN)
    print(f"--- FINISHED LoRA RUN: r={r_val} ---")

    # Save adapter to Drive
    run_name_lora = run_config_lora.get('run_name')
    colab_adapter_path_lora = f"./results/{run_name_lora}/best_adapter"
    drive_adapter_path_lora = os.path.join(DRIVE_SAVE_PATH_BASE, run_name_lora)
    print(f"Copying adapter from {colab_adapter_path_lora} to {drive_adapter_path_lora}...")
    !cp -r "{colab_adapter_path_lora}" "{drive_adapter_path_lora}"
    print("Adapter copied to Google Drive.")


--- STARTING LoRA RUN: r=32 ---


--- Loading dataset: samsum ---
--- Loading SUBSET of 5000 training samples ---
Using 5000 training samples.
--- Loading SUBSET of 200 evaluation samples ---
Using 200 eval samples out of 818.
--- Loading model: t5 ---
--- Applying PEFT adapter: lora ---
trainable params: 1,179,648 || all params: 61,686,272 || trainable%: 1.9123
--- Using Seq2SeqTrainer (for eval_loss only) ---
--- Starting Training: t5_samsum_lora_r32 ---


/content/lora-vs-ia3/src/trainer.py:173: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = TrainerClass(


Epoch,Training Loss,Validation Loss
1,0.473200,0.388716
2,0.465800,0.378074
3,0.470100,0.376091


--- Starting Evaluation (calculating eval_loss) ---


Evaluation Results: {'eval_loss': 0.37609124183654785, 'eval_runtime': 9.1351, 'eval_samples_per_second': 21.894, 'eval_steps_per_second': 10.947, 'epoch': 3.0}
--- Saving Adapter ---
Best adapter saved to ./results/t5_samsum_lora_r32/best_adapter


eval/loss,█▂▁▁
eval/runtime,▇▄█▁
eval/samples_per_second,▂▄▁█
eval/steps_per_second,▂▄▁█
train/epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇████
train/global_step,▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇███
train/grad_norm,▇█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▂▃▅████▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▁▁▁▁
train/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,3
eval/loss,0.37609


--- FINISHED LoRA RUN: r=32 ---
Copying adapter from ./results/t5_samsum_lora_r32/best_adapter to /content/drive/MyDrive/lora-vs-ia3/adapters/t5_samsum_lora_r32...
Adapter copied to Google Drive.


In [ ]:
for r_val in [64]:
    run_config_lora = base_config.copy()
    run_config_lora.update({
        "run_name": f"t5_samsum_lora_r{r_val}",
        "peft_method": "lora",
        "r": r_val,
        "lora_alpha": r_val * 2,
    })

    print(f"\n--- STARTING LoRA RUN: r={r_val} ---")
    run_experiment(run_config_lora, HF_TOKEN)
    print(f"--- FINISHED LoRA RUN: r={r_val} ---")

    # Save adapter to Drive
    run_name_lora = run_config_lora.get('run_name')
    colab_adapter_path_lora = f"./results/{run_name_lora}/best_adapter"
    drive_adapter_path_lora = os.path.join(DRIVE_SAVE_PATH_BASE, run_name_lora)
    print(f"Copying adapter from {colab_adapter_path_lora} to {drive_adapter_path_lora}...")
    !cp -r "{colab_adapter_path_lora}" "{drive_adapter_path_lora}"
    print("Adapter copied to Google Drive.")


--- STARTING LoRA RUN: r=64 ---


train/epoch,▁
train/global_step,▁
train/grad_norm,▁
train/learning_rate,▁
train/loss,▁
total_params,60531200
train/epoch,0.032
train/global_step,10
train/grad_norm,2.69571
train/learning_rate,9e-05
train/loss,4.0518


--- Loading dataset: samsum ---
--- Loading SUBSET of 5000 training samples ---
Using 5000 training samples.
--- Loading SUBSET of 200 evaluation samples ---
Using 200 eval samples out of 818.
--- Loading model: t5 ---
--- Applying PEFT adapter: lora ---
trainable params: 2,359,296 || all params: 62,865,920 || trainable%: 3.7529
--- Using Seq2SeqTrainer (for eval_loss only) ---
--- Starting Training: t5_samsum_lora_r64 ---


/content/lora-vs-ia3/src/trainer.py:173: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = TrainerClass(


Epoch,Training Loss,Validation Loss
1,0.468300,0.382782
2,0.460500,0.371126
3,0.458800,0.368873


--- Starting Evaluation (calculating eval_loss) ---


Evaluation Results: {'eval_loss': 0.3688732087612152, 'eval_runtime': 9.2813, 'eval_samples_per_second': 21.549, 'eval_steps_per_second': 10.774, 'epoch': 3.0}
--- Saving Adapter ---
Best adapter saved to ./results/t5_samsum_lora_r64/best_adapter


eval/loss,█▂▁▁
eval/runtime,██▅▁
eval/samples_per_second,▁▁▃█
eval/steps_per_second,▁▁▃█
train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,█▃▃▂▃▄▇▁▁▃▄▂▆▁▃▅▃▂▃▄▄▂▇▃▃▁▄▇▄▂▃▅▃▄▄▃▁▅▅▅
train/learning_rate,▂▆████▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▁▁▁▁
train/loss,█▄▄▃▃▃▄▂▃▃▃▂▁▂▂▂▂▃▂▂▂▁▃▂▂▂▃▃▂▂▂▂▂▁▃▂▂▁▂▁
epoch,3
eval/loss,0.36887


--- FINISHED LoRA RUN: r=64 ---
Copying adapter from ./results/t5_samsum_lora_r64/best_adapter to /content/drive/MyDrive/lora-vs-ia3/adapters/t5_samsum_lora_r64...
Adapter copied to Google Drive.


In [ ]:
for r_val in [128]:
    run_config_lora = base_config.copy()
    run_config_lora.update({
        "run_name": f"t5_samsum_lora_r{r_val}",
        "peft_method": "lora",
        "r": r_val,
        "lora_alpha": r_val * 2,
    })

    print(f"\n--- STARTING LoRA RUN: r={r_val} ---")
    run_experiment(run_config_lora, HF_TOKEN)
    print(f"--- FINISHED LoRA RUN: r={r_val} ---")

    # Save adapter to Drive
    run_name_lora = run_config_lora.get('run_name')
    colab_adapter_path_lora = f"./results/{run_name_lora}/best_adapter"
    drive_adapter_path_lora = os.path.join(DRIVE_SAVE_PATH_BASE, run_name_lora)
    print(f"Copying adapter from {colab_adapter_path_lora} to {drive_adapter_path_lora}...")
    !cp -r "{colab_adapter_path_lora}" "{drive_adapter_path_lora}"
    print("Adapter copied to Google Drive.")


--- STARTING LoRA RUN: r=128 ---


--- Loading dataset: samsum ---
--- Loading SUBSET of 5000 training samples ---
Using 5000 training samples.
--- Loading SUBSET of 200 evaluation samples ---
Using 200 eval samples out of 818.
--- Loading model: t5 ---
--- Applying PEFT adapter: lora ---
trainable params: 4,718,592 || all params: 65,225,216 || trainable%: 7.2343
--- Using Seq2SeqTrainer (for eval_loss only) ---
--- Starting Training: t5_samsum_lora_r128 ---


/content/lora-vs-ia3/src/trainer.py:173: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = TrainerClass(


Epoch,Training Loss,Validation Loss
1,0.461100,0.377164
2,0.450800,0.365516
3,0.445600,0.362406


--- Starting Evaluation (calculating eval_loss) ---


Evaluation Results: {'eval_loss': 0.3624063730239868, 'eval_runtime': 9.7618, 'eval_samples_per_second': 20.488, 'eval_steps_per_second': 10.244, 'epoch': 3.0}
--- Saving Adapter ---
Best adapter saved to ./results/t5_samsum_lora_r128/best_adapter


eval/loss,█▂▁▁
eval/runtime,█▁█▄
eval/samples_per_second,▁█▁▄
eval/steps_per_second,▁█▁▄
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
train/global_step,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇███
train/grad_norm,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▅█████▇▇▇▇▆▆▆▆▆▆▆▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁
train/loss,█▂▂▂▂▂▂▁▁▁▂▂▁▁▂▁▁▁▁▁▁▂▁▁▂▁▂▁▁▁▁▂▁▁▁▁▁▁▁▁
epoch,3
eval/loss,0.36241


--- FINISHED LoRA RUN: r=128 ---
Copying adapter from ./results/t5_samsum_lora_r128/best_adapter to /content/drive/MyDrive/lora-vs-ia3/adapters/t5_samsum_lora_r128...
Adapter copied to Google Drive.


In [ ]:
run_config_ia3 = base_config.copy()
run_config_ia3.update({
    "run_name": "t5_samsum_ia3",
    "peft_method": "ia3",
})

print(f"\n--- STARTING IA³ RUN ---")
run_experiment(run_config_ia3, HF_TOKEN)
print(f"--- FINISHED IA³ RUN ---")


--- STARTING IA³ RUN ---


--- Loading dataset: samsum ---
--- Loading SUBSET of 5000 training samples ---


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss
1,0.619100,0.504975
2,0.595100,0.473128
3,0.600900,0.465238


--- Starting Evaluation (calculating eval_loss) ---


Evaluation Results: {'eval_loss': 0.4652383327484131, 'eval_runtime': 8.9408, 'eval_samples_per_second': 22.369, 'eval_steps_per_second': 11.185, 'epoch': 3.0}
--- Saving Adapter ---
Best adapter saved to ./results/t5_samsum_ia3/best_adapter


eval/loss,█▂▁▁
eval/runtime,█▁█▁
eval/samples_per_second,▁█▁█
eval/steps_per_second,▁█▁█
train/epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇██
train/global_step,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇████
train/grad_norm,▄▄▄█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,███▇▇▇▇▇▇▆▆▆▅▅▅▄▄▄▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁
train/loss,██▆▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,3
eval/loss,0.46524


--- FINISHED IA³ RUN ---
